# Fire Prediction - Sentinel Hub API Integration

This notebook demonstrates how to extract metadata from Sentinel-2 XML files and integrate with the Sentinel Hub API for fire prediction analysis.

## Objectives:
- Extract essential metadata from XML files (coordinates, dates, tile info)
- Transform coordinates from UTM to WGS84
- Create Sentinel Hub API requests for pre/post fire imagery
- Prepare data for fire damage analysis

## Requirements:
- Valid Sentinel Hub credentials (CLIENT_ID and CLIENT_SECRET) in `.env` file
- XML metadata files from Sentinel-2 imagery
- Python libraries: sentinelhub, pyproj, shapely

## Usage:
1. Run all cells in sequence
2. Check the summary section for pipeline status
3. Use the created API requests to download satellite data

In [32]:
# Sentinel Hub API Testing for Fire Prediction
# Metadata extraction from XML files and Sentinel Hub API integration

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import datetime as dt

# Data handling and visualization
import matplotlib.pyplot as plt
import numpy as np

# Geospatial libraries
import pyproj
from shapely.geometry import box
from shapely.ops import transform

# Sentinel Hub API
from sentinelhub import CRS, BBox, DataCollection, MimeType, SentinelHubRequest, SHConfig, generate_evalscript
from sentinelhub.api.catalog import get_available_timestamps

# Cloud detection (optional)
from s2cloudless import S2PixelCloudDetector

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print(" All libraries imported successfully!")

 All libraries imported successfully!


In [28]:
# Configuration Setup for Sentinel Hub API

# Load credentials from environment variables
CLIENT_ID = os.getenv('CLIENT_ID', None)
CLIENT_SECRET = os.getenv('CLIENT_SECRET', None)

# Initialize Sentinel Hub configuration
config = SHConfig()

# Set credentials if available
if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET
    print("Sentinel Hub credentials configured successfully!")
    print(f"Client ID: {CLIENT_ID[:8]}...")
else:
    print("   Sentinel Hub credentials not found in environment variables")
    print("   Set CLIENT_ID and CLIENT_SECRET in your .env file to use the API")
    print("   You can still use the metadata extraction functionality")

print("\nConfiguration ready!")

Sentinel Hub credentials configured successfully!
Client ID: 26aa2248...

Configuration ready!


# 1. Metadata Extraction Classes

This section contains the core classes for extracting metadata from Sentinel-2 XML files and integrating with the Sentinel Hub API.

##  Features:
- **MetadataExtractor**: Extracts essential data from XML metadata files
- **SentinelHubDataExtractor**: Creates API requests from extracted metadata
- **Coordinate transformation**: UTM to WGS84 conversion
- **Fire analysis support**: Pre/post fire data extraction

In [30]:
class MetadataExtractor:
    """
    A robust class to extract essential metadata from Sentinel-2 XML files
    for use with Sentinel Hub API
    """
    
    def __init__(self, xml_folder_path: str):
        """
        Initialize the MetadataExtractor
        
        Args:
            xml_folder_path (str): Path to the folder containing XML files
        """
        self.xml_folder_path = Path(xml_folder_path)
        self.metadata_cache = {}
    
    def parse_xml_file(self, xml_file_path: str, verbose: bool = False) -> Dict:
        """
        Parse a single XML metadata file and extract essential information for Sentinel Hub API
        
        Args:
            xml_file_path (str): Path to the XML file
            verbose (bool): Print detailed parsing information
            
        Returns:
            Dict: Extracted metadata information
        """
        try:
            tree = ET.parse(xml_file_path)
            root = tree.getroot()
            
            # Define namespace
            namespaces = {
                'n1': 'https://psd-14.sentinel2.eo.esa.int/PSD/S2_PDI_Level-1C_Tile_Metadata.xsd'
            }
            
            # Helper function to find elements with fallback methods
            def find_element(xpath_with_ns, xpath_without_ns=None):
                element = root.find(xpath_with_ns, namespaces)
                if element is not None:
                    return element
                if xpath_without_ns:
                    return root.find(xpath_without_ns)
                simple_xpath = xpath_with_ns.replace('n1:', '').replace('//', '.')
                return root.find(f".//{simple_xpath}")
            
            # Extract essential information
            tile_id_elem = find_element('.//n1:TILE_ID', './/TILE_ID')
            sensing_time_elem = find_element('.//n1:SENSING_TIME', './/SENSING_TIME')
            cs_name_elem = find_element('.//n1:HORIZONTAL_CS_NAME', './/HORIZONTAL_CS_NAME')
            cs_code_elem = find_element('.//n1:HORIZONTAL_CS_CODE', './/HORIZONTAL_CS_CODE')
            
            # Geometric information for 10m resolution
            geoposition_10m = find_element('.//n1:Geoposition[@resolution="10"]', './/Geoposition[@resolution="10"]')
            size_10m = find_element('.//n1:Size[@resolution="10"]', './/Size[@resolution="10"]')
            
            # Extract coordinate values
            ulx_val = uly_val = xdim_val = ydim_val = None
            if geoposition_10m is not None:
                ulx_elem = geoposition_10m.find('ULX') if geoposition_10m.find('ULX') is not None else geoposition_10m.find('.//ULX')
                uly_elem = geoposition_10m.find('ULY') if geoposition_10m.find('ULY') is not None else geoposition_10m.find('.//ULY')
                xdim_elem = geoposition_10m.find('XDIM') if geoposition_10m.find('XDIM') is not None else geoposition_10m.find('.//XDIM')
                ydim_elem = geoposition_10m.find('YDIM') if geoposition_10m.find('YDIM') is not None else geoposition_10m.find('.//YDIM')
                
                ulx_val = float(ulx_elem.text) if ulx_elem is not None else None
                uly_val = float(uly_elem.text) if uly_elem is not None else None
                xdim_val = float(xdim_elem.text) if xdim_elem is not None else None
                ydim_val = float(ydim_elem.text) if ydim_elem is not None else None
            
            # Extract size values
            nrows_val = ncols_val = None
            if size_10m is not None:
                nrows_elem = size_10m.find('NROWS') if size_10m.find('NROWS') is not None else size_10m.find('.//NROWS')
                ncols_elem = size_10m.find('NCOLS') if size_10m.find('NCOLS') is not None else size_10m.find('.//NCOLS')
                
                nrows_val = int(nrows_elem.text) if nrows_elem is not None else None
                ncols_val = int(ncols_elem.text) if ncols_elem is not None else None
            
            # Create metadata dictionary
            metadata = {
                'file_path': xml_file_path,
                'tile_id': tile_id_elem.text if tile_id_elem is not None else None,
                'sensing_time': sensing_time_elem.text if sensing_time_elem is not None else None,
                'coordinate_system_name': cs_name_elem.text if cs_name_elem is not None else None,
                'coordinate_system_code': cs_code_elem.text if cs_code_elem is not None else None,
                'upper_left_x': ulx_val,
                'upper_left_y': uly_val,
                'x_dimension': xdim_val,
                'y_dimension': ydim_val,
                'rows': nrows_val,
                'cols': ncols_val,
            }
            
            if verbose and metadata['tile_id']:
                print(f"{Path(xml_file_path).name}: {metadata['tile_id']}")
            
            return metadata
            
        except Exception as e:
            if verbose:
                print(f"Error parsing {Path(xml_file_path).name}: {str(e)}")
            return {}
    
    def get_bounding_box_wgs84(self, metadata: Dict) -> Optional[Tuple[float, float, float, float]]:
        """
        Calculate bounding box in WGS84 coordinates from UTM metadata
        
        Args:
            metadata (Dict): Metadata dictionary from parse_xml_file
            
        Returns:
            Tuple: (min_lon, min_lat, max_lon, max_lat) or None if conversion fails
        """
        try:
            required_keys = ['upper_left_x', 'upper_left_y', 'x_dimension', 'y_dimension', 'rows', 'cols']
            
            if not all(key in metadata and metadata[key] is not None for key in required_keys):
                return None
            
            ulx = metadata['upper_left_x']
            uly = metadata['upper_left_y']
            x_dim = abs(metadata['x_dimension'])
            y_dim = abs(metadata['y_dimension'])
            rows = metadata['rows']
            cols = metadata['cols']
            
            # Calculate corner coordinates in UTM
            min_x, max_x = ulx, ulx + (cols * x_dim)
            max_y, min_y = uly, uly - (rows * y_dim)
            
            # Extract EPSG code and transform coordinates
            epsg_code = metadata['coordinate_system_code']
            if epsg_code and epsg_code.startswith('EPSG:'):
                epsg_num = int(epsg_code.split(':')[1])
                
                utm_crs = pyproj.CRS(f'EPSG:{epsg_num}')
                wgs84_crs = pyproj.CRS('EPSG:4326')
                transformer = pyproj.Transformer.from_crs(utm_crs, wgs84_crs, always_xy=True)
                
                min_lon, min_lat = transformer.transform(min_x, min_y)
                max_lon, max_lat = transformer.transform(max_x, max_y)
                
                return (min_lon, min_lat, max_lon, max_lat)
            
        except Exception:
            pass
            
        return None
    
    def extract_all_metadata(self, verbose: bool = True) -> List[Dict]:
        """
        Extract metadata from all XML files in the folder
        
        Args:
            verbose (bool): Print progress information
            
        Returns:
            List[Dict]: List of metadata dictionaries
        """
        all_metadata = []
        
        xml_files = list(self.xml_folder_path.glob("*metadata*.xml"))
        if verbose:
            print(f"📁 Found {len(xml_files)} metadata XML files")
        
        for xml_file in xml_files:
            metadata = self.parse_xml_file(str(xml_file), verbose=False)
            
            if metadata and metadata.get('tile_id'):
                # Add file info
                metadata['file_name'] = xml_file.name
                metadata['location'] = self.extract_location_from_filename(xml_file.name)
                metadata['time_period'] = self.extract_time_period_from_filename(xml_file.name)
                
                # Calculate WGS84 bounding box
                bbox_wgs84 = self.get_bounding_box_wgs84(metadata)
                if bbox_wgs84:
                    metadata['bbox_wgs84'] = bbox_wgs84
                
                all_metadata.append(metadata)
        
        if verbose:
            print(f"Successfully extracted metadata from {len(all_metadata)} files")
        return all_metadata
    
    def extract_location_from_filename(self, filename: str) -> str:
        """Extract location name from filename"""
        parts = filename.replace('.xml', '').split('_')
        return parts[0] if parts else 'unknown'
    
    def extract_time_period_from_filename(self, filename: str) -> str:
        """Extract time period (pre/post) from filename"""
        if 'pre' in filename:
            return 'pre'
        elif 'post' in filename:
            return 'post'
        return 'unknown'
    
    def get_metadata_for_api(self, metadata: Dict) -> Dict:
        """
        Extract only the essential data needed for Sentinel Hub API calls
        
        Args:
            metadata (Dict): Full metadata dictionary
            
        Returns:
            Dict: Simplified metadata for API use
        """
        api_metadata = {
            'location': metadata.get('location'),
            'time_period': metadata.get('time_period'),
            'sensing_time': metadata.get('sensing_time'),
            'tile_id': metadata.get('tile_id'),
        }
        
        if 'bbox_wgs84' in metadata:
            bbox = metadata['bbox_wgs84']
            api_metadata['bbox'] = {
                'min_lon': bbox[0],
                'min_lat': bbox[1],
                'max_lon': bbox[2],
                'max_lat': bbox[3]
            }
        
        return api_metadata

# Initialize the metadata extractor
xml_folder_path = "/Users/diego/Documents/FirePrediction/data_pipeline/utils/data_api/testing/copied_xml_files"
extractor = MetadataExtractor(xml_folder_path)

print("MetadataExtractor initialized successfully!")

MetadataExtractor initialized successfully!


## Sentinel Hub API Integration

Integration class for creating API requests from extracted metadata.

In [23]:
class SentinelHubDataExtractor:
    """
    A class to extract data from Sentinel Hub API using metadata from XML files
    """
    
    def __init__(self, config: SHConfig):
        """
        Initialize the SentinelHubDataExtractor
        
        Args:
            config (SHConfig): Sentinel Hub configuration
        """
        self.config = config
    
    def create_bbox_from_metadata(self, metadata: Dict, buffer_km: float = 1.0) -> BBox:
        """
        Create a BBox object from metadata with optional buffer
        
        Args:
            metadata (Dict): Metadata dictionary containing bbox_wgs84
            buffer_km (float): Buffer distance in kilometers
            
        Returns:
            BBox: Sentinel Hub BBox object
        """
        if 'bbox_wgs84' not in metadata:
            raise ValueError("No WGS84 bounding box found in metadata")
        
        min_lon, min_lat, max_lon, max_lat = metadata['bbox_wgs84']
        
        # Add buffer (approximately 1km = 0.009 degrees)
        buffer_deg = buffer_km * 0.009
        
        bbox = BBox(
            bbox=[min_lon - buffer_deg, min_lat - buffer_deg, 
                  max_lon + buffer_deg, max_lat + buffer_deg],
            crs=CRS.WGS84
        )
        
        return bbox
    
    def get_evalscript_rgb(self) -> str:
        """
        Generate evalscript for RGB visualization
        
        Returns:
            str: Evalscript for RGB bands
        """
        return """
        //VERSION=3
        function setup() {
            return {
                input: ["B02", "B03", "B04"],
                output: { bands: 3 }
            };
        }
        
        function evaluatePixel(sample) {
            return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
        }
        """
    
    def get_evalscript_ndvi(self) -> str:
        """
        Generate evalscript for NDVI calculation
        
        Returns:
            str: Evalscript for NDVI
        """
        return """
        //VERSION=3
        function setup() {
            return {
                input: ["B04", "B08"],
                output: { bands: 1, sampleType: "FLOAT32" }
            };
        }
        
        function evaluatePixel(sample) {
            let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
            return [ndvi];
        }
        """
    
    def get_evalscript_all_bands(self) -> str:
        """
        Generate evalscript for all Sentinel-2 bands
        
        Returns:
            str: Evalscript for all bands
        """
        return """
        //VERSION=3
        function setup() {
            return {
                input: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"],
                output: { bands: 12, sampleType: "UINT16" }
            };
        }
        
        function evaluatePixel(sample) {
            return [sample.B01, sample.B02, sample.B03, sample.B04, 
                   sample.B05, sample.B06, sample.B07, sample.B08,
                   sample.B8A, sample.B09, sample.B11, sample.B12];
        }
        """
    
    def create_request(self, metadata: Dict, evalscript: str, 
                      time_range: Tuple[str, str], resolution: int = 10,
                      image_format: MimeType = MimeType.TIFF) -> SentinelHubRequest:
        """
        Create a Sentinel Hub request based on metadata
        
        Args:
            metadata (Dict): Metadata from XML file
            evalscript (str): Evalscript for data processing
            time_range (Tuple[str, str]): Time range as (start_date, end_date)
            resolution (int): Output resolution in meters
            image_format (MimeType): Output image format
            
        Returns:
            SentinelHubRequest: Configured request object
        """
        bbox = self.create_bbox_from_metadata(metadata)
        
        request = SentinelHubRequest(
            evalscript=evalscript,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L1C,
                    time_interval=time_range,
                    mosaicking_order='leastCC'  # Least cloud coverage
                )
            ],
            responses=[
                SentinelHubRequest.output_response('default', image_format)
            ],
            bbox=bbox,
            size=[512, 512],  # Can be adjusted based on needs
            config=self.config
        )
        
        return request
    
    def extract_data_for_location(self, location_metadata: List[Dict], 
                                 evalscript_type: str = 'rgb',
                                 days_buffer: int = 30) -> Dict:
        """
        Extract data for a specific location (pre and post fire)
        
        Args:
            location_metadata (List[Dict]): List of metadata for the same location
            evalscript_type (str): Type of evalscript ('rgb', 'ndvi', 'all_bands')
            days_buffer (int): Number of days to search around the sensing date
            
        Returns:
            Dict: Dictionary containing pre and post fire data
        """
        # Get evalscript based on type
        if evalscript_type == 'rgb':
            evalscript = self.get_evalscript_rgb()
        elif evalscript_type == 'ndvi':
            evalscript = self.get_evalscript_ndvi()
        elif evalscript_type == 'all_bands':
            evalscript = self.get_evalscript_all_bands()
        else:
            raise ValueError(f"Unknown evalscript type: {evalscript_type}")
        
        results = {}
        
        for metadata in location_metadata:
            if metadata['sensing_time']:
                # Parse sensing time and create time range
                sensing_date = dt.datetime.fromisoformat(metadata['sensing_time'].replace('Z', '+00:00'))
                start_date = (sensing_date - dt.timedelta(days=days_buffer)).strftime('%Y-%m-%d')
                end_date = (sensing_date + dt.timedelta(days=days_buffer)).strftime('%Y-%m-%d')
                
                # Create request
                request = self.create_request(
                    metadata=metadata,
                    evalscript=evalscript,
                    time_range=(start_date, end_date)
                )
                
                # Store request for the time period
                time_period = metadata.get('time_period', 'unknown')
                results[f"{time_period}_request"] = request
                results[f"{time_period}_metadata"] = metadata
        
        return results

# Initialize the Sentinel Hub data extractor
if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET
    
    data_extractor = SentinelHubDataExtractor(config)
    print("Sentinel Hub data extractor initialized successfully!")
else:
    print("Warning: CLIENT_ID and CLIENT_SECRET not found. Please set them in your .env file.")
    print("You can still use the metadata extraction functionality.")

Sentinel Hub data extractor initialized successfully!


# 2. Practical Examples

This section demonstrates how to use the metadata extractor and Sentinel Hub API integration.

In [31]:
# Extract metadata from all XML files and organize by location
print("Extracting metadata from XML files...")
all_metadata = extractor.extract_all_metadata(verbose=True)

# Group metadata by location
locations = {}
for metadata in all_metadata:
    location = metadata['location']
    if location not in locations:
        locations[location] = []
    locations[location].append(metadata)

print(f"\nFound {len(locations)} unique locations:")
for location, metadata_list in locations.items():
    pre_count = sum(1 for m in metadata_list if m['time_period'] == 'pre')
    post_count = sum(1 for m in metadata_list if m['time_period'] == 'post')
    print(f"   {location}: {pre_count} pre-fire, {post_count} post-fire")

# Show sample metadata
if all_metadata:
    sample = all_metadata[0]
    print(f"\nSample metadata:")
    print(f"   Location: {sample['location']}")
    print(f"   Time: {sample['sensing_time']}")
    print(f"   Tile: {sample['tile_id']}")
    if 'bbox_wgs84' in sample:
        bbox = sample['bbox_wgs84']
        print(f"   Area: {bbox[0]:.3f}°, {bbox[1]:.3f}° to {bbox[2]:.3f}°, {bbox[3]:.3f}°")

print(f"\nMetadata extraction complete!")

Extracting metadata from XML files...
📁 Found 24 metadata XML files
Successfully extracted metadata from 24 files

Found 12 unique locations:
   usa: 1 pre-fire, 1 post-fire
   chile: 1 pre-fire, 1 post-fire
   sardinia: 1 pre-fire, 1 post-fire
   greece2: 1 pre-fire, 1 post-fire
   spain: 1 pre-fire, 1 post-fire
   usa2: 1 pre-fire, 1 post-fire
   turkey: 1 pre-fire, 1 post-fire
   france: 1 pre-fire, 1 post-fire
   spain2: 1 pre-fire, 1 post-fire
   spain3: 1 pre-fire, 1 post-fire
   greece: 1 pre-fire, 1 post-fire
   paraguay: 1 pre-fire, 1 post-fire

Sample metadata:
   Location: usa
   Time: 2025-07-15T18:24:31.742376Z
   Tile: S2C_OPER_MSI_L1C_TL_2CPS_20250715T214706_A004487_T12SUF_N05.11
   Area: -113.220°, 36.036° to -112.014°, 37.042°

Metadata extraction complete!


## API Integration Example

Creating Sentinel Hub API requests from extracted metadata.

In [ ]:
# Create Sentinel Hub API requests from metadata

def create_sentinel_request_from_metadata(metadata_dict, evalscript_type='rgb', days_buffer=15):
    """
    Create a Sentinel Hub request using extracted metadata
    
    Args:
        metadata_dict: Dictionary with extracted metadata
        evalscript_type: Type of data to extract ('rgb', 'ndvi', 'all_bands')
        days_buffer: Days before/after sensing time to search
    
    Returns:
        SentinelHubRequest object ready for execution
    """
    
    # Validate required fields
    required_fields = ['sensing_time', 'bbox_wgs84']
    missing_fields = [field for field in required_fields if field not in metadata_dict]
    if missing_fields:
        raise ValueError(f"Missing required fields: {missing_fields}")
    
    # Create bounding box
    bbox_coords = metadata_dict['bbox_wgs84']
    bbox = BBox(bbox=bbox_coords, crs=CRS.WGS84)
    
    # Parse sensing time and create time interval
    sensing_time = dt.datetime.fromisoformat(metadata_dict['sensing_time'].replace('Z', '+00:00'))
    start_date = (sensing_time - dt.timedelta(days=days_buffer)).strftime('%Y-%m-%d')
    end_date = (sensing_time + dt.timedelta(days=days_buffer)).strftime('%Y-%m-%d')
    
    # Choose evalscript based on type
    evalscripts = {
        'rgb': '''
        //VERSION=3
        function setup() {
            return {
                input: ["B02", "B03", "B04"],
                output: { bands: 3 }
            };
        }
        function evaluatePixel(sample) {
            return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
        }
        ''',
        'ndvi': '''
        //VERSION=3
        function setup() {
            return {
                input: ["B04", "B08"],
                output: { bands: 1, sampleType: "FLOAT32" }
            };
        }
        function evaluatePixel(sample) {
            let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
            return [ndvi];
        }
        ''',
        'all_bands': '''
        //VERSION=3
        function setup() {
            return {
                input: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"],
                output: { bands: 12, sampleType: "UINT16" }
            };
        }
        function evaluatePixel(sample) {
            return [sample.B01, sample.B02, sample.B03, sample.B04, 
                   sample.B05, sample.B06, sample.B07, sample.B08,
                   sample.B8A, sample.B09, sample.B11, sample.B12];
        }
        '''
    }
    
    # evalscript = evalscripts.get(evalscript_type, evalscripts['rgb'])

    data_collection = DataCollection.SENTINEL2_L1C

    evalscript = generate_evalscript(
        data_collection=data_collection,
        meta_bands=["dataMask"],
        merged_bands_output="bands",
        prioritize_dn=False,
    )
    
    # Create the request
    request = SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection,
                time_interval=(start_date, end_date)
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.TIFF),
            SentinelHubRequest.output_response('dataMask', MimeType.TIFF),
        ],
        bbox=bbox,
        resolution=(10,10),  # 10m resolution
        config=config,
    )
    
    return request

# Demonstrate API usage with available locations
print("🚀 Demonstrating Sentinel Hub API integration...")

if CLIENT_ID and CLIENT_SECRET and locations:
    # Find a location with both pre and post data
    complete_locations = []
    for location, metadata_list in locations.items():
        pre_data = [m for m in metadata_list if m['time_period'] == 'pre' and 'bbox_wgs84' in m]
        post_data = [m for m in metadata_list if m['time_period'] == 'post' and 'bbox_wgs84' in m]
        
        if pre_data and post_data:
            complete_locations.append((location, pre_data[0], post_data[0]))
    
    if complete_locations:
        location, pre_metadata, post_metadata = complete_locations[0]
        
        print(f"📍 Creating API requests for {location}:")
        print(f"   Pre-fire:  {pre_metadata['sensing_time']}")
        print(f"   Post-fire: {post_metadata['sensing_time']}")
        
        try:
            # Create requests for RGB data
            pre_rgb_request = create_sentinel_request_from_metadata(pre_metadata, 'rgb')
            post_rgb_request = create_sentinel_request_from_metadata(post_metadata, 'rgb')
            
            print(f"✅ API requests created successfully!")
            print(f"   Ready to execute: pre_rgb_data = pre_rgb_request.get_data()")
            print(f"   Ready to execute: post_rgb_data = post_rgb_request.get_data()")
            
            # Store for potential use
            globals()['pre_rgb_request'] = pre_rgb_request
            globals()['post_rgb_request'] = post_rgb_request
            
        except Exception as e:
            print(f"❌ Error creating requests: {e}")
    else:
        print("⚠️  No locations with complete pre/post fire data found")
        
elif not (CLIENT_ID and CLIENT_SECRET):
    print("⚠️  Sentinel Hub credentials not configured")
    print("   Set CLIENT_ID and CLIENT_SECRET in .env file to test API functionality")
else:
    print("⚠️  No metadata extracted yet - run the previous cell first")

print(f"\n🎯 API integration ready for fire analysis!")

# 3. Summary and Next Steps

Final summary of the pipeline status and next steps for fire analysis.

In [ ]:
# Summary and Next Steps for Fire Analysis

print("🔥 FIRE PREDICTION DATA PIPELINE - SUMMARY")
print("=" * 60)

# Show what we have accomplished
if 'all_metadata' in globals() and all_metadata:
    print(f"✅ XML metadata extraction: {len(all_metadata)} files processed")
    
    if 'locations' in globals():
        complete_pairs = []
        for location, metadata_list in locations.items():
            pre_data = [m for m in metadata_list if m['time_period'] == 'pre' and 'bbox_wgs84' in m]
            post_data = [m for m in metadata_list if m['time_period'] == 'post' and 'bbox_wgs84' in m]
            if pre_data and post_data:
                complete_pairs.append(location)
        
        print(f"✅ Location analysis: {len(locations)} locations found")
        print(f"✅ Complete fire pairs: {len(complete_pairs)} locations ready")
        print(f"   Locations with pre/post data: {', '.join(complete_pairs)}")
    
    if CLIENT_ID and CLIENT_SECRET:
        print(f"✅ Sentinel Hub API: Configured and ready")
        if 'pre_rgb_request' in globals():
            print(f"✅ API requests: Created and ready to execute")
        else:
            print(f"⚠️  API requests: Not yet created")
    else:
        print(f"⚠️  Sentinel Hub API: Credentials not configured")
    
    print(f"\n📊 NEXT STEPS FOR FIRE ANALYSIS:")
    print(f"   1. Execute API requests to download satellite data")
    print(f"   2. Compare pre/post fire imagery")
    print(f"   3. Calculate burn severity using NDVI")
    print(f"   4. Train fire prediction models")
    
    print(f"\n📝 EXAMPLE USAGE:")
    if 'pre_rgb_request' in globals():
        print(f"   # Download pre-fire RGB data")
        print(f"   pre_rgb_data = pre_rgb_request.get_data()")
        print(f"   ")
        print(f"   # Download post-fire RGB data")
        print(f"   post_rgb_data = post_rgb_request.get_data()")
        print(f"   ")
        print(f"   # Analyze differences")
        print(f"   difference = post_rgb_data - pre_rgb_data")
    else:
        print(f"   # First run the previous cells to create API requests")
        print(f"   # Then execute the requests to download data")
    
else:
    print(f"❌ Run the previous cells to extract metadata first")

print(f"\n🎯 PIPELINE STATUS: Ready for fire damage analysis!")
print("=" * 60)

=== COMPLETE FIRE PREDICTION DATA EXTRACTION WORKFLOW ===
Step 1: Extract metadata from all XML files...
Scanning for XML files in: /Users/diego/Documents/FirePrediction/data_pipeline/utils/data_api/testing/copied_xml_files
Found 24 metadata XML files

Processing: usa_post_metadata.xml
Successfully parsed: usa_post_metadata.xml
  Tile ID: S2C_OPER_MSI_L1C_TL_2CPS_20250715T214706_A004487_T12SUF_N05.11
  Sensing Time: 2025-07-15T18:24:31.742376Z
  Coordinate System: EPSG:32612
  Upper Left: (300000.0, 4100040.0)
  Bounding box calculated: (-113.21991719169252, 36.036167697666194, -112.01430253846226, 37.04224724177572)

Processing: chile_post_metadata.xml
Successfully parsed: chile_post_metadata.xml
  Tile ID: S2A_OPER_MSI_L1C_TL_2APS_20240212T175807_A045138_T19HBD_N05.10
  Sensing Time: 2024-02-12T14:52:04.975566Z
  Coordinate System: EPSG:32719
  Upper Left: (199980.0, 6400000.0)
  Bounding box calculated: (-72.22879766580088, -33.48569245991556, -71.0252199246242, -32.521046863655314)

/var/folders/mr/n19psg616pb4dyc16wbyn8yc0000gn/T/ipykernel_42847/1187624985.py:77: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  ulx_elem = geoposition_10m.find('.//ULX') or geoposition_10m.find('ULX')
/var/folders/mr/n19psg616pb4dyc16wbyn8yc0000gn/T/ipykernel_42847/1187624985.py:78: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  uly_elem = geoposition_10m.find('.//ULY') or geoposition_10m.find('ULY')
/var/folders/mr/n19psg616pb4dyc16wbyn8yc0000gn/T/ipykernel_42847/1187624985.py:79: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  xdim_elem = geoposition_10m.find('.//XDIM') or geoposition_10m.find('XDIM')
/var/folders/mr/n19psg616pb4dyc16wbyn8yc0000gn